<a href="https://colab.research.google.com/github/Jaehwi-So/DeepLearning_Study/blob/main/DL07_NLP_Seq2Seq_%EB%B2%88%EC%97%AD%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('drive/MyDrive/DL2024_201810776/week12')

%load_ext autoreload
%autoreload 2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# 1. 데이터 읽고 전처리
- 영어와 프랑스어가 매칭되어있는 데이터셋을 사용하였다.
- src에 영어, tar에 프랑스어가 매칭되어 있는 구조이다.
- https://www.kaggle.com/code/jasoncallaway/fra-txt-details

In [3]:
lines = pd.read_csv('dataset/fra.txt', names=['src', 'tar', 'lic'], sep='\t')
del lines['lic']
print('전체 샘플의 개수 :',len(lines))

전체 샘플의 개수 : 232736


In [4]:
lines = lines.loc[:, 'src':'tar']
lines = lines[0:30000] # 5만개만 사용
lines.sample(10)

,src,tar
2882,Be cheerful.,Soyez heureux.
15503,I have the list.,J'ai la liste.
15183,How large is it?,C'est grand comment ?
22912,That's not a lie.,Ce n'est pas un mensonge.
19549,Do I look thirty?,Ai-je l'air d'avoir trente ans ?
2888,Be creative.,Soyez créatif !
8153,I trusted you.,Je vous faisais confiance.
25492,Can you go faster?,Es-tu en mesure d'aller plus vite ?
16597,It's a dead end.,C'est un cul-de-sac.
9772,"Tom, hurry up.","Tom, dépêche-toi."


In [5]:
lines.tar = lines.tar.apply(lambda x : '\t '+ x + ' \n')
lines.sample(10)

,src,tar
16196,I'm feeling fit.,\t Je me sens en forme. \n
20858,I hope you agree.,\t J'espère que tu es d'accord. \n
5368,I must study.,\t Je dois étudier. \n
11156,I am off today.,"\t Je ne suis pas de service, aujourd'hui. \n"
6388,They'll call.,\t Ils appelleront. \n
13094,This is a fact.,\t C'est un fait. \n
22,Who?,\t Qui ? \n
94,Get up.,\t Lève-toi. \n
4627,You're nuts!,\t Vous êtes givré ! \n
25832,Don't forget that.,\t N'oublie pas cela. \n


###  문자 집합 구축하기
데이터셋으로부터 문자 단위로 Set을 구축한다

In [6]:
src_vocab = set()
for line in lines.src: # 1줄씩 읽음
    for char in line: # 1개의 문자씩 읽음
        src_vocab.add(char)

tar_vocab = set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)
src_vocab_size = len(src_vocab)+1
tar_vocab_size = len(tar_vocab)+1
print('source 문장의 char 집합 :',src_vocab_size)
print('target 문장의 char 집합 :',tar_vocab_size)


source 문장의 char 집합 : 77
target 문장의 char 집합 : 102


In [7]:
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))
print(src_vocab[45:75])
print(tar_vocab[45:75])


['W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['V', 'W', 'X', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [8]:
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'X': 47, 'Y': 48, 'Z': 49, 'a': 50, 'b': 51, 'c': 52, 'd': 53, 'e': 54, 'f': 55, 'g': 56, 'h': 57, 'i': 58, 'j': 59, 'k': 60, 'l': 61, 'm': 62, 'n': 63, 'o': 64, 'p': 65, 'q': 66, 'r': 67, 's': 68, 't': 69, 'u': 70, 'v': 71, 'w': 72, 'x': 73, 'y': 74, 'z': 75, 'é': 76}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '$': 6, '%': 7, '&': 8, "'": 9, ',': 10, '-': 11, '.': 12, '0': 13, '1': 14, '2': 15, '3': 16, '4': 17, '5': 18, '6': 19, '7': 20, '8': 21, '9': 22, ':': 23, '?': 24, 'A': 25, 'B': 26, 'C': 27, 'D': 28, 'E': 29, 'F': 30, 'G': 31, 'H': 32, 'I': 33, 'J': 34, 'K': 35, 'L': 36, 'M': 3

# Character Embedding
새로운 단어와 희귀한 단어가 자주 등장하므로 Character Embedding을 택하였고, 문장을 문자 단위로 인코딩해주자.

In [9]:
encoder_input = []

# 1개의 문장
for line in lines.src:
  encoded_line = []
  # 각 줄에서 1개의 char (Tom came after me)
  for char in line:
    # 각 char을 정수로 변환 (T, o, m, ,c ...)
    encoded_line.append(src_to_index[char])
  encoder_input.append(encoded_line)
print('원래 source 문장 :', lines.src[:5])
print('source 문장의 정수 인코딩 :',encoder_input[:5])


원래 source 문장 : 0    Go.
1    Go.
2    Go.
3    Go.
4    Hi.
Name: src, dtype: object
source 문장의 정수 인코딩 : [[30, 64, 10], [30, 64, 10], [30, 64, 10], [30, 64, 10], [31, 58, 10]]


In [10]:
decoder_input = []
for line in lines.tar:
  encoded_line = []
  for char in line:
    encoded_line.append(tar_to_index[char])
  decoder_input.append(encoded_line)
print('target 문장의 정수 인코딩 :',decoder_input[:5])


target 문장의 정수 인코딩 : [[1, 3, 46, 50, 3, 4, 3, 2], [1, 3, 37, 50, 67, 52, 57, 54, 12, 3, 2], [1, 3, 29, 63, 3, 67, 64, 70, 69, 54, 3, 4, 3, 2], [1, 3, 26, 64, 70, 56, 54, 3, 4, 3, 2], [1, 3, 43, 50, 61, 70, 69, 3, 4, 3, 2]]


In [11]:
decoder_target = []
for line in lines.tar:
  timestep = 0
  encoded_line = []
  for char in line:
    if timestep > 0:
      encoded_line.append(tar_to_index[char])
    timestep = timestep + 1
  decoder_target.append(encoded_line)
print('target 문장 레이블의 정수 인코딩 :',decoder_target[:5])


target 문장 레이블의 정수 인코딩 : [[3, 46, 50, 3, 4, 3, 2], [3, 37, 50, 67, 52, 57, 54, 12, 3, 2], [3, 29, 63, 3, 67, 64, 70, 69, 54, 3, 4, 3, 2], [3, 26, 64, 70, 56, 54, 3, 4, 3, 2], [3, 43, 50, 61, 70, 69, 3, 4, 3, 2]]


In [12]:
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print('source 문장의 최대 길이 :',max_src_len)
print('target 문장의 최대 길이 :',max_tar_len)

source 문장의 최대 길이 : 18
target 문장의 최대 길이 : 61


### 입력 크기를 통일시키기 위해서 패딩을 추가해준다.

In [13]:
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

<br><hr><br>

# 2. 모델 생성

In [14]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, Attention
from tensorflow.keras.models import Model
import numpy as np

### 2-1. Encoder
Encoder의 LSTM에는 return_state를 설정하고 return_sequence를 설정하지 않는다(출력이 필요하지 않기 때문)

In [15]:
encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True) # Encoder은 return_state를 설정한 LSTM 선언
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs) # Encoder 출력 (encoder_outputs, state_h, state_c)에 저장
encoder_states = [state_h, state_c] # LSTM은 상태가 두 개. 은닉 상태와 셀 상태.

### 2-2. Decoder
- Decoder LSTM에는 return_state와 함께 return_sequence를 설정하여 출력 시퀀스를 받을 수 있도록 한다.
- 출력에 대해서 softmax를 적용하여 확률이 높은 결과를 출력하도록 한다.

In [16]:
decoder_inputs = Input(shape=(None, tar_vocab_size))
# Decoder은 return_state와 return sequence를 설정한 LSTM 선언
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)

# Decoder에게 Encoder의 은닉 상태, 셀 상태를 전달.
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)

decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
seq2seq_outputs = decoder_softmax_layer(decoder_outputs)


### 2-3 Seq2Seq

In [17]:
#모델 선언
model = Model([encoder_inputs, decoder_inputs], seq2seq_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

In [18]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None, 77)]           0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, None, 102)]          0         []                            
                                                                                                  
 lstm (LSTM)                 [(None, 256),                342016    ['input_1[0][0]']             
                              (None, 256),                                                        
                              (None, 256)]                                                        
                                                                                              

<br><hr><br>

# 3. 모델 학습

In [20]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=40, validation_split=0.2)

Epoch 1/40
375/375 [==============================] - 6s 16ms/step - loss: 0.4566 - val_loss: 0.5638
Epoch 2/40
375/375 [==============================] - 5s 13ms/step - loss: 0.4405 - val_loss: 0.5488
Epoch 3/40
375/375 [==============================] - 5s 13ms/step - loss: 0.4267 - val_loss: 0.5319
Epoch 4/40
375/375 [==============================] - 7s 18ms/step - loss: 0.4141 - val_loss: 0.5188
Epoch 5/40
375/375 [==============================] - 5s 13ms/step - loss: 0.4027 - val_loss: 0.5110
Epoch 6/40
375/375 [==============================] - 6s 15ms/step - loss: 0.3921 - val_loss: 0.5039
Epoch 7/40
375/375 [==============================] - 5s 14ms/step - loss: 0.3825 - val_loss: 0.4929
Epoch 8/40
375/375 [==============================] - 5s 13ms/step - loss: 0.3736 - val_loss: 0.4818
Epoch 9/40
375/375 [==============================] - 6s 16ms/step - loss: 0.3655 - val_loss: 0.4753
Epoch 10/40
375/375 [==============================] - 5s 13ms/step - loss: 0.3577 - val_lo

<br><hr><br>

# 4. 모델 결과 확인
훈련된 모델을 사용해서 입력 시퀀스에 대한 번역을 생성하기 위한 추론 모델을 구성한다.
- encoder_model: 인코더 모델을 구성. 인코더의 입력을 주면 인코더의 상태를 출력
- decoder_model: 디코더 모델을 구성. 디코더의 초기 상태와 입력을 주면 디코더의 출력과 상태를 출력

In [21]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [22]:
# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용
# decoder_lstm을 사용하면서 state_h, state_c 같이 저장하기
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)


# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태를 버리지 않음.
decoder_states = [state_h, state_c]
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [23]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())


### 번역 생성
decode_sequence: 주어진 입력 시퀀스에 대해 번역을 생성하는 함수.
- 인코더를 통해 초기 상태를 얻고, 그 상태를 디코더의 초기 상태로 사용하여 디코더를 반복적으로 실행하여 출력을 생성한다.

In [24]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 상태를 얻음
  states_value = encoder_model.predict(input_seq)

  # <SOS>에 해당하는 원-핫 벡터 생성
  target_seq = np.zeros((1, 1, tar_vocab_size))
  target_seq[0, 0, tar_to_index['\t']] = 1.

  stop_condition = False
  decoded_sentence = ""

  # stop_condition이 True가 될 때까지 루프 반복
  while not stop_condition:
    # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    # 예측 결과를 문자로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]

    # 현재 시점의 예측 문자를 예측 문장에 추가
    decoded_sentence += sampled_char

    # <eos>에 도달하거나 최대 길이를 넘으면 중단.
    if (sampled_char == '\n' or
        len(decoded_sentence) > max_tar_len):
        stop_condition = True

    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, sampled_token_index] = 1.

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

  return decoded_sentence


### 번역 결과 확인
(대충 엉성하지만 번역 자체가 된다는 것을 의의로 두자! 이정도만 해서 잘 되는게 이상하다!)

In [25]:
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스
  input_seq = encoder_input[seq_index:seq_index+1]
  decoded_sentence = decode_sequence(input_seq)
  print(35 * "-")
  print('입력 문장:', lines.src[seq_index])
  print('정답 문장:', lines.tar[seq_index][2:len(lines.tar[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
  print('번역 문장:', decoded_sentence[1:len(decoded_sentence)-1]) # '\n'을 빼고 출력


1/1 [==============================] - 0s 29ms/step
-----------------------------------
입력 문장: Go.
정답 문장: Bouge ! 
번역 문장: Continuez ! 
1/1 [==============================] - 0s 25ms/step
-----------------------------------
입력 문장: Hello!
정답 문장: Bonjour ! 
번역 문장: Soyez prudente ! 
1/1 [==============================] - 0s 24ms/step
-----------------------------------
입력 문장: Got it!
정답 문장: J'ai pigé ! 
번역 문장: Comme courigue ! 
1/1 [==============================] - 0s 24ms/step
-----------------------------------
입력 문장: Go home.
정답 문장: Rentre à la maison. 
번역 문장: Allez chercher ! 
1/1 [==============================] - 0s 38ms/step
-----------------------------------
입력 문장: Forget me.
정답 문장: Oublie-moi. 
번역 문장: Oubliez-le. 
